In [ ]:
#Install Tensorflow

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization,LeakyReLU
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
import matplotlib.pyplot as plt
%matplotlib inline 
# To enable plotting graphs in Jupyter notebook
import seaborn as sns

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [ ]:
project_path = '/content/drive/My Drive/Colab Notebooks/Project6/'

In [ ]:
dataset_file = project_path + 'bank.csv'

In [ ]:
data = pd.read_csv(dataset_file)

In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#It seems RowNumber, CustomerID, Surname are just redundant not anyways useful here 
#Was thinking of removing IsActiveMember but can be a possibility it is used for some prediction

In [ ]:
data = data.drop(["RowNumber","CustomerId","Surname"], axis = 1)

In [ ]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#Check for any null or missig values

In [ ]:
data.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [ ]:
#No missing values found
#Get the basic infor of the fields 

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [ ]:
#it seems Geography and Gender are object fields
#Lets look at the values of both and make sure should we hot encode it or delete it 

In [ ]:
data.Geography.unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [ ]:
data.Gender.unique()

array(['Female', 'Male'], dtype=object)

In [ ]:
#Lets look at the correlation matrix

In [ ]:
data.corr()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
CreditScore,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097
Exited,-0.027094,0.285323,-0.014001,0.118533,-0.047820,-0.007138,-0.156128,0.012097,1.000000


In [ ]:
data.shape

(10000, 11)

In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
CreditScore,10000.0,650.528800,96.653299,350.00,584.00,652.000,718.0000,850.00
Age,10000.0,38.921800,10.487806,18.00,32.00,37.000,44.0000,92.00
Tenure,10000.0,5.012800,2.892174,0.00,3.00,5.000,7.0000,10.00
Balance,10000.0,76485.889288,62397.405202,0.00,0.00,97198.540,127644.2400,250898.09
NumOfProducts,10000.0,1.530200,0.581654,1.00,1.00,1.000,2.0000,4.00
HasCrCard,10000.0,0.705500,0.455840,0.00,0.00,1.000,1.0000,1.00
IsActiveMember,10000.0,0.515100,0.499797,0.00,0.00,1.000,1.0000,1.00
EstimatedSalary,10000.0,100090.239881,57510.492818,11.58,51002.11,100193.915,149388.2475,199992.48
Exited,10000.0,0.203700,0.402769,0.00,0.00,0.000,0.0000,1.00


In [ ]:
data.skew()

CreditScore       -0.071607
Age                1.011320
Tenure             0.010991
Balance           -0.141109
NumOfProducts      0.745568
HasCrCard         -0.901812
IsActiveMember    -0.060437
EstimatedSalary    0.002085
Exited             1.471611
dtype: float64

In [ ]:
#As we can see nothing is highly skewed from above

In [ ]:
pd.crosstab(data['Geography'], data['Exited'], normalize='index')

Exited,0,1
Geography,,
France,0.838452,0.161548
Germany,0.675568,0.324432
Spain,0.833266,0.166734


In [ ]:
pd.crosstab(data['Gender'], data['Exited'], normalize='index')

Exited,0,1
Gender,,
Female,0.749285,0.250715
Male,0.835441,0.164559


In [ ]:
#From above it seems People in germany are more likely to exit and females overall are more likely to exit 


In [ ]:
#Lets replace male as 1 and female as 0 and germany as 1 and other countries as 0 

In [ ]:
data.Gender.replace({'Female': 0, 'Male': 1},inplace=True)

In [ ]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
data.Geography.replace({'France': 0,'Spain': 0, 'Germany': 1},inplace=True)

In [ ]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,0,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,0,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
data.corr()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
CreditScore,1.000000,0.005538,-0.002857,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Geography,0.005538,1.000000,-0.024628,0.046897,-0.000567,0.401110,-0.010419,0.010577,-0.020486,0.010297,0.173488
Gender,-0.002857,-0.024628,1.000000,-0.027544,0.014733,0.012087,-0.021859,0.005766,0.022544,-0.008112,-0.106512
Age,-0.003965,0.046897,-0.027544,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,0.000842,-0.000567,0.014733,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,0.006268,0.401110,0.012087,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.012238,-0.010419,-0.021859,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,-0.005458,0.010577,0.005766,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.025651,-0.020486,0.022544,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.001384,0.010297,-0.008112,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097


In [ ]:
#From above it seems isActiveMember,Tenure,EstimatedSalary,NumOfProducts,CreditScore,HasCrCard has very minimum correlation with the Exited. 
#IsActiveMember has little higher correlation but it is almost synonymous to exited so we can remove that column

In [ ]:
data = data.drop(["IsActiveMember","Tenure","EstimatedSalary","NumOfProducts","CreditScore","HasCrCard"], axis = 1)

In [ ]:
data.head()

,Geography,Gender,Age,Balance,Exited
0,0,0,42,0.00,1
1,0,0,41,83807.86,0
2,0,0,42,159660.80,1
3,0,0,39,0.00,0
4,0,0,43,125510.82,0


In [ ]:
#Split Data 

In [ ]:
X_data = data.iloc[:, :-1]

In [ ]:
X_data.shape

(10000, 4)

In [ ]:
y_data = data.iloc[:, -1]

In [ ]:
y_data.shape

(10000,)

In [ ]:
#Divide in to test and train data with 30% in test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.3, random_state = 7)

In [ ]:
#Lets Normalize the data
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
# MinMaxScalar has been used here. You can go ahead and use the other scalars available and chcek the effect on the results.
#fitting the transform on test and train separately
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train


array([[0.        , 0.        , 0.24324324, 0.46317216],
       [1.        , 1.        , 0.27027027, 0.23030149],
       [1.        , 0.        , 0.44594595, 0.50229874],
       ...,
       [0.        , 1.        , 0.22972973, 0.56996053],
       [0.        , 1.        , 0.21621622, 0.        ],
       [0.        , 1.        , 0.10810811, 0.41396518]])

In [ ]:
# let us now convert the data elements into tensors as we need tensors to be fed into different tensorflow based operations
#X-train and X_test were converted to numpy arrays while transformations while the other two need to be transformed into numpy arrays.
X_train=tf.convert_to_tensor(X_train)
y_train=tf.convert_to_tensor(y_train.values)
X_test=tf.convert_to_tensor(X_test)
y_test=tf.convert_to_tensor(y_test.values)

In [ ]:
X_train.shape[1]

4

In [ ]:
#Creating a model Sequential with relu, tanh and sigmoid

In [ ]:
model = Sequential()

In [ ]:
#Adding layers

In [ ]:
model.add(Dense(64, input_shape = (4,), activation = 'relu'))
model.add(Dense(32, activation = 'tanh'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
#Model compile [optimizers and loss functions]

In [ ]:
sgd = optimizers.Adam(lr = 0.003)

In [ ]:
model.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, batch_size = 50, epochs = 50, verbose = 1)

Epoch 1/50
140/140 [==============================] - 0s 1ms/step - loss: 0.4837 - accuracy: 0.7920
Epoch 2/50
140/140 [==============================] - 0s 1ms/step - loss: 0.4455 - accuracy: 0.7973
Epoch 3/50
140/140 [==============================] - 0s 1ms/step - loss: 0.4412 - accuracy: 0.7980
Epoch 4/50
140/140 [==============================] - 0s 1ms/step - loss: 0.4373 - accuracy: 0.8044
Epoch 5/50
140/140 [==============================] - 0s 1ms/step - loss: 0.4350 - accuracy: 0.8044
Epoch 6/50
140/140 [==============================] - 0s 1ms/step - loss: 0.4340 - accuracy: 0.8070
Epoch 7/50
140/140 [==============================] - 0s 1ms/step - loss: 0.4325 - accuracy: 0.8070
Epoch 8/50
140/140 [==============================] - 0s 1ms/step - loss: 0.4321 - accuracy: 0.8073
Epoch 9/50
140/140 [==============================] - 0s 1ms/step - loss: 0.4294 - accuracy: 0.8120
Epoch 10/50
140/140 [==============================] - 0s 1ms/step - loss: 0.4289 - accuracy: 0.8111

In [ ]:
#Evaluation

In [ ]:
results = model.evaluate(X_test, y_test)

94/94 [==============================] - 0s 821us/step - loss: 0.4111 - accuracy: 0.8170


In [ ]:
print(model.metrics_names)
print(results)    

['loss', 'accuracy']
[0.4111209213733673, 0.8169999718666077]


In [ ]:
Y_predict = model.predict(X_test, batch_size=250, verbose=0)
Y_pred = Y_predict >0.5
print('Accuracy Model New (Dropout): '+ str(model.evaluate(X_test,y_test)[1]))
print('Recall_score: ' + str(recall_score(y_test,Y_pred)))
print('Precision_score: ' + str(precision_score(y_test, Y_pred,zero_division=1)))
print('F-score: ' + str(f1_score(y_test,Y_pred,zero_division=1)))

confusion_matrix(y_test, Y_pred)

94/94 [==============================] - 0s 843us/step - loss: 0.4111 - accuracy: 0.8170
Accuracy Model New (Dropout): 0.8169999718666077
Recall_score: 0.35537190082644626
Precision_score: 0.5748663101604278
F-score: 0.43922369765066394


array([[2236,  159],
       [ 390,  215]])

In [ ]:
#Creating a another Mode

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.3, random_state = 7)

In [ ]:
#Lets Normalize the data
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
# MinMaxScalar has been used here. You can go ahead and use the other scalars available and chcek the effect on the results.
#fitting the transform on test and train separately
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# let us now convert the data elements into tensors as we need tensors to be fed into different tensorflow based operations
#X-train and X_test were converted to numpy arrays while transformations while the other two need to be transformed into numpy arrays.
X_train=tf.convert_to_tensor(X_train)
y_train=tf.convert_to_tensor(y_train.values)
X_test=tf.convert_to_tensor(X_test)
y_test=tf.convert_to_tensor(y_test.values)

In [ ]:
model_new = Sequential()

In [ ]:
model_new.add(Dense(128, input_shape = (4,), activation = 'relu'))
model_new.add(Dense(64, activation = 'tanh'))
model_new.add(Dense(1, activation = 'sigmoid'))

In [ ]:
sgd_new = optimizers.Adamax(lr = 0.001)

In [ ]:
model_new.compile(optimizer = sgd_new, loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_new.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               640       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 8,961
Trainable params: 8,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_new.fit(X_train, y_train, batch_size = 250, epochs = 50, verbose = 1)

Epoch 1/50
28/28 [==============================] - 0s 2ms/step - loss: 0.5627 - accuracy: 0.7841
Epoch 2/50
28/28 [==============================] - 0s 1ms/step - loss: 0.5114 - accuracy: 0.7954
Epoch 3/50
28/28 [==============================] - 0s 2ms/step - loss: 0.4970 - accuracy: 0.7954
Epoch 4/50
28/28 [==============================] - 0s 1ms/step - loss: 0.4873 - accuracy: 0.7954
Epoch 5/50
28/28 [==============================] - 0s 1ms/step - loss: 0.4795 - accuracy: 0.7954
Epoch 6/50
28/28 [==============================] - 0s 1ms/step - loss: 0.4723 - accuracy: 0.7946
Epoch 7/50
28/28 [==============================] - 0s 1ms/step - loss: 0.4658 - accuracy: 0.7954
Epoch 8/50
28/28 [==============================] - 0s 1ms/step - loss: 0.4605 - accuracy: 0.7979
Epoch 9/50
28/28 [==============================] - 0s 1ms/step - loss: 0.4562 - accuracy: 0.7986
Epoch 10/50
28/28 [==============================] - 0s 1ms/step - loss: 0.4522 - accuracy: 0.7977
Epoch 11/50
28/28 [

In [ ]:
results = model_new.evaluate(X_test, y_test)

94/94 [==============================] - 0s 839us/step - loss: 0.4277 - accuracy: 0.8150


In [ ]:
print(model_new.metrics_names)
print(results)    

['loss', 'accuracy']
[0.42773717641830444, 0.8149999976158142]


In [ ]:
Y_predict = model_new.predict(X_test, batch_size=250, verbose=0)
Y_pred = Y_predict >0.5
print('Accuracy Model New (Dropout): '+ str(model_new.evaluate(X_test,y_test)[1]))
print('Recall_score: ' + str(recall_score(y_test,Y_pred)))
print('Precision_score: ' + str(precision_score(y_test, Y_pred,zero_division=1)))
print('F-score: ' + str(f1_score(y_test,Y_pred,zero_division=1)))

confusion_matrix(y_test, Y_pred)

94/94 [==============================] - 0s 981us/step - loss: 0.4277 - accuracy: 0.8150
Accuracy Model New (Dropout): 0.8149999976158142
Recall_score: 0.20991735537190082
Precision_score: 0.6225490196078431
F-score: 0.31396786155747836


array([[2318,   77],
       [ 478,  127]])

In [ ]:
#Trying with ADAM and ADAMX we get better accuracy score with adam and also the recall_score which is suppose to be closer to 1 is better in ADAM with 1, 32 and 64 
#This shows the model has accurately predicted 81% of the user but since the recall_score is not closer to 1 we will have to manipulate more and look out for other activation factors


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.3, random_state = 7)

In [ ]:
#Lets Normalize the data
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
# MinMaxScalar has been used here. You can go ahead and use the other scalars available and chcek the effect on the results.
#fitting the transform on test and train separately
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# let us now convert the data elements into tensors as we need tensors to be fed into different tensorflow based operations
#X-train and X_test were converted to numpy arrays while transformations while the other two need to be transformed into numpy arrays.
X_train=tf.convert_to_tensor(X_train)
y_train=tf.convert_to_tensor(y_train.values)
X_test=tf.convert_to_tensor(X_test)
y_test=tf.convert_to_tensor(y_test.values)

In [ ]:
model_softmax = Sequential()

In [ ]:
model_softmax.add(Dense(256, input_shape = (4,), activation = 'relu'))
model_softmax.add(Dense(128, activation = 'tanh'))
model_softmax.add(Dense(1, activation='softmax'))



In [ ]:
sgd_softmax = optimizers.Adam(lr = 0.005)

In [ ]:
model_softmax.compile(optimizer = sgd_softmax, loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model_softmax.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 256)               1280      
_________________________________________________________________
dense_82 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_83 (Dense)             (None, 1)                 129       
Total params: 34,305
Trainable params: 34,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_softmax.fit(X_train, y_train, batch_size = 250, epochs = 50, verbose = 1)

Epoch 1/50
28/28 [==============================] - 0s 3ms/step - loss: 12.1297 - accuracy: 0.2046
Epoch 2/50
28/28 [==============================] - 0s 3ms/step - loss: 12.1297 - accuracy: 0.2046
Epoch 3/50
28/28 [==============================] - 0s 3ms/step - loss: 12.1297 - accuracy: 0.2046
Epoch 4/50
28/28 [==============================] - 0s 3ms/step - loss: 12.1297 - accuracy: 0.2046
Epoch 5/50
28/28 [==============================] - 0s 3ms/step - loss: 12.1297 - accuracy: 0.2046
Epoch 6/50
28/28 [==============================] - 0s 3ms/step - loss: 12.1297 - accuracy: 0.2046
Epoch 7/50
28/28 [==============================] - 0s 3ms/step - loss: 12.1297 - accuracy: 0.2046
Epoch 8/50
28/28 [==============================] - 0s 3ms/step - loss: 12.1297 - accuracy: 0.2046
Epoch 9/50
28/28 [==============================] - 0s 3ms/step - loss: 12.1297 - accuracy: 0.2046
Epoch 10/50
28/28 [==============================] - 0s 3ms/step - loss: 12.1297 - accuracy: 0.2046
Epoch 11/

In [ ]:
results = model_softmax.evaluate(X_test, y_test)

94/94 [==============================] - 0s 889us/step - loss: 12.1740 - accuracy: 0.2017


In [ ]:
print(model_softmax.metrics_names)
print(results)    

['loss', 'accuracy']
[12.173974990844727, 0.2016666680574417]


In [ ]:
Y_predict = model_softmax.predict(X_test, batch_size=250, verbose=0)
print(Y_predict)
Y_pred = Y_predict >0.5
print('Accuracy Model New (Dropout): '+ str(model_softmax.evaluate(X_test,y_test)[1]))
print('Recall_score: ' + str(recall_score(y_test,Y_predict)))
print('Precision_score: ' + str(precision_score(y_test, Y_predict,zero_division=1)))
print('F-score: ' + str(f1_score(y_test,Y_predict,zero_division=1)))

confusion_matrix(y_test, Y_predict)

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
94/94 [==============================] - 0s 911us/step - loss: 12.1740 - accuracy: 0.2017
Accuracy Model New (Dropout): 0.2016666680574417
Recall_score: 1.0
Precision_score: 0.20166666666666666
F-score: 0.3356449375866852


array([[   0, 2395],
       [   0,  605]])

In [ ]:
#With Softmax activation as well we dont get a great accuracy model